In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

Datasets in [GOPRO](https://seungjunnah.github.io/Datasets/gopro.html)
1. Download GOPRO_Large_all
2. Download GOPRO_Large
3. mkdir a data folder
4. unzip GOPRO_Large_all under data
5. unzip GOPRO_Large under data
6. root = './data/'  
The dataset under root is as follows. 
* GOPRO_Large_all
* GOPRO_Large
* GTMMP
 * train (synthetic blurry frame from GOPRO_Large_all)
 * test (test from GOPRO_Large)
 * mmp_train (generate using optical flow from RAFT)
 * mmp_test (generate using optical flow from RAFT)

In [ ]:
root = './data/'
root = 'E:/gopro/'

## Generate ground truth MMP

Optical flow from RAFT

Average of adjescent optical flow, back and forth  
normalized by the maximum value of one sequence

In [3]:
import sys
sys.path.append('core')

import argparse
import os
import cv2
import glob
import numpy as np
import torch
from PIL import Image

from raft import RAFT
from utils import flow_viz
from utils.utils import InputPadder

DEVICE = 'cuda'

In [4]:
pathlist=[]
pathlist.append('GTMMP/mmp_train/GOPR0372_07_00/') #7 61
pathlist.append('GTMMP/mmp_train/GOPR0372_07_01/') #7 59
pathlist.append('GTMMP/mmp_train/GOPR0374_11_00/') #11 116
pathlist.append('GTMMP/mmp_train/GOPR0374_11_01/') #11 108
pathlist.append('GTMMP/mmp_train/GOPR0374_11_02/') #11 154
pathlist.append('GTMMP/mmp_train/GOPR0374_11_03/') #11 125
pathlist.append('GTMMP/mmp_train/GOPR0378_13_00/') #13 40
pathlist.append('GTMMP/mmp_train/GOPR0379_11_00/') #11 67
pathlist.append('GTMMP/mmp_train/GOPR0380_11_00/') #11 121
pathlist.append('GTMMP/mmp_train/GOPR0384_11_01/') #11 100
pathlist.append('GTMMP/mmp_train/GOPR0384_11_02/') #11 62
pathlist.append('GTMMP/mmp_train/GOPR0384_11_03/') #11 86
pathlist.append('GTMMP/mmp_train/GOPR0384_11_04/') #11 150
pathlist.append('GTMMP/mmp_train/GOPR0385_11_00/') #11 64
pathlist.append('GTMMP/mmp_train/GOPR0386_11_00/') #11 107
pathlist.append('GTMMP/mmp_train/GOPR0477_11_00/') #11 155
pathlist.append('GTMMP/mmp_train/GOPR0857_11_00/') #11 54 
pathlist.append('GTMMP/mmp_train/GOPR0868_11_01/') #11 46
pathlist.append('GTMMP/mmp_train/GOPR0868_11_02/') #11 64
pathlist.append('GTMMP/mmp_train/GOPR0871_11_01/') #11 65
pathlist.append('GTMMP/mmp_train/GOPR0881_11_00/') #11 84
pathlist.append('GTMMP/mmp_train/GOPR0884_11_00/') #11

In [5]:
import os
for name in pathlist:
    folder_name = root + name
    if os.path.exists(folder_name) is False:
        os.makedirs(folder_name)

In [6]:
pathlist=[]
pathlist.append(['GTMMP/mmp_test/GOPR0384_11_00/',11])
pathlist.append(['GTMMP/mmp_test/GOPR0384_11_05/',11])
pathlist.append(['GTMMP/mmp_test/GOPR0385_11_01/',11])
pathlist.append(['GTMMP/mmp_test/GOPR0396_11_00/',11])
pathlist.append(['GTMMP/mmp_test/GOPR0410_11_00/',11])
pathlist.append(['GTMMP/mmp_test/GOPR0854_11_00/',11])
pathlist.append(['GTMMP/mmp_test/GOPR0862_11_00/',11])
pathlist.append(['GTMMP/mmp_test/GOPR0868_11_00/',11])
pathlist.append(['GTMMP/mmp_test/GOPR0869_11_00/',11])
pathlist.append(['GTMMP/mmp_test/GOPR0871_11_00/',11])
pathlist.append(['GTMMP/mmp_test/GOPR0881_11_01/',11])

In [7]:
import os
for name,frames in pathlist:
    folder_name = root + name
    if os.path.exists(folder_name) is False:
        os.makedirs(folder_name)

In [9]:
pathlist=[]
#path to gopro_large_all, frames used to generate a synthetic frame in original gopro dataset
pathlist.append(['GOPRO_Large_all/train/GOPR0372_07_00/',7]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0372_07_01/',7]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0374_11_00/',11])
pathlist.append(['GOPRO_Large_all/train/GOPR0374_11_01/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0374_11_02/',11])
pathlist.append(['GOPRO_Large_all/train/GOPR0374_11_03/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0378_13_00/',13]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0379_11_00/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0380_11_00/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0384_11_01/',11])
pathlist.append(['GOPRO_Large_all/train/GOPR0384_11_02/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0384_11_03/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0384_11_04/',11])
pathlist.append(['GOPRO_Large_all/train/GOPR0385_11_00/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0386_11_00/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0477_11_00/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0857_11_00/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0868_11_01/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0868_11_02/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0871_11_01/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0881_11_00/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0884_11_00/',11]) 

In [11]:
def load_image(imfile):
    img = np.array(Image.open(imfile)).astype(np.uint8)
    img = torch.from_numpy(img).permute(2, 0, 1).float()
    return img[None].to(DEVICE)

In [ ]:
epochs = 20
for name in pathlist:
    path = root + name[0]
    print(path)
    parser = argparse.ArgumentParser()
    parser.add_argument('--model',default='raft-things.pth', help="restore checkpoint")
    parser.add_argument('--path',default=path, help="dataset for evaluation")
    parser.add_argument('--small', action='store_true', help='use small model')
    parser.add_argument('--mixed_precision', action='store_true', help='use mixed precision')
    parser.add_argument('--alternate_corr', action='store_true', help='use efficent correlation implementation')
    args = parser.parse_args(args=[])
    print(args)
    model = torch.nn.DataParallel(RAFT(args))
    model.load_state_dict(torch.load(args.model))

    model = model.module
    model.to(DEVICE)
    model.eval()
    avg_max=torch.tensor([0]).cuda()
    with torch.no_grad():
        images = glob.glob(os.path.join(args.path, '*.png')) + \
                    glob.glob(os.path.join(args.path, '*.jpg'))
        images = sorted(images)
        frames = name[1]
        for i in range(0,len(images)-frames):
            bl_avg=0
            for j in range(frames-1):
                image1 = load_image(images[i+j])
                imagen = load_image(images[i+j+1]) #10
        
                padder = InputPadder(image1.shape)
                image1, imagen = padder.pad(image1, imagen)
           
                flow_low_1n, flow_up_1n = model(image1, imagen, iters=epochs, test_mode=True)
            
                bl_1n=torch.sqrt(flow_up_1n[:,0,:,:]**2+flow_up_1n[:,1,:,:]**2)
            
                if j==0:
                    bl_avg += 2*bl_1n
                else:
                    bl_avg += bl_1n
            for j in reversed(range(1,frames)):
                image1 = load_image(images[i+j])
                imagen = load_image(images[i+j-1]) 
        
                padder = InputPadder(image1.shape)
                image1, imagen = padder.pad(image1, imagen)
           
                flow_low_1n, flow_up_1n = model(image1, imagen, iters=epochs, test_mode=True)
            
                bl_1n=torch.sqrt(flow_up_1n[:,0,:,:]**2+flow_up_1n[:,1,:,:]**2)
            
                if j==frames-1:
                    bl_avg += 2*bl_1n
                else:
                    bl_avg += bl_1n
            bl=bl_avg.cpu().numpy().squeeze()/2/frames
       
            save_path=images[i].replace("GOPRO_Large_all/train","GTMMP/mmp_train")
        
            save_path = save_path.replace("png","npy")
            
            out = bl/15  #K value
            
            out[out>1]=1
            
            out = out.astype(np.float32)
            
            np.save(save_path, out)
        
            if np.max(bl)>avg_max:
                avg_max=np.max(bl)

      
    print(avg_max)

In [ ]:
pathlist=[]
pathlist.append(['GOPRO_Large_all/test/GOPR0384_11_00/',11,5])
pathlist.append(['GOPRO_Large_all/test/GOPR0384_11_05/',11,15])
pathlist.append(['GOPRO_Large_all/test/GOPR0385_11_01/',11,10])
pathlist.append(['GOPRO_Large_all/test/GOPR0396_11_00/',11,3])
pathlist.append(['GOPRO_Large_all/test/GOPR0410_11_00/',11,19])
pathlist.append(['GOPRO_Large_all/test/GOPR0854_11_00/',11,7])
pathlist.append(['GOPRO_Large_all/test/GOPR0862_11_00/',11,7])
pathlist.append(['GOPRO_Large_all/test/GOPR0868_11_00/',11,5])
pathlist.append(['GOPRO_Large_all/test/GOPR0869_11_00/',11,10])
pathlist.append(['GOPRO_Large_all/test/GOPR0871_11_00/',11,8])
pathlist.append(['GOPRO_Large_all/test/GOPR0881_11_01/',11,9])

In [ ]:
epochs = 20
for name in pathlist:
    path = name[0]
    print(path)
    parser = argparse.ArgumentParser()
    parser.add_argument('--model',default='raft-things.pth', help="restore checkpoint")
    parser.add_argument('--path',default=path, help="dataset for evaluation")
    parser.add_argument('--small', action='store_true', help='use small model')
    parser.add_argument('--mixed_precision', action='store_true', help='use mixed precision')
    parser.add_argument('--alternate_corr', action='store_true', help='use efficent correlation implementation')
    args = parser.parse_args(args=[])
    print(args)
    model = torch.nn.DataParallel(RAFT(args))
    model.load_state_dict(torch.load(args.model))

    model = model.module
    model.to(DEVICE)
    model.eval()
    avg_max=torch.tensor([0]).cuda()
    with torch.no_grad():
        images = glob.glob(os.path.join(args.path, '*.png')) + \
                    glob.glob(os.path.join(args.path, '*.jpg'))
        images = sorted(images)
        frames = name[1]
        #for i in range(len(images)-7):
        for i in range(0,len(images),frames):
            bl_avg=0
            for j in range(frames-1):
                image1 = load_image(images[i+j])
                imagen = load_image(images[i+j+1]) #10
        
                padder = InputPadder(image1.shape)
                image1, imagen = padder.pad(image1, imagen)
           
                flow_low_1n, flow_up_1n = model(image1, imagen, iters=epochs, test_mode=True)
            
                bl_1n=torch.sqrt(flow_up_1n[:,0,:,:]**2+flow_up_1n[:,1,:,:]**2)
            
                #bl_avg=(bl_12+bl_23+bl_34+bl_45+bl_56+bl_67)   #/7.0  +bl_78
                if j==0:
                    bl_avg += 2*bl_1n
                else:
                    bl_avg += bl_1n
            for j in reversed(range(1,frames)):
                #print(1)
                image1 = load_image(images[i+j])
                imagen = load_image(images[i+j-1]) #10
        
                padder = InputPadder(image1.shape)
                image1, imagen = padder.pad(image1, imagen)
           
                flow_low_1n, flow_up_1n = model(image1, imagen, iters=epochs, test_mode=True)
            
                bl_1n=torch.sqrt(flow_up_1n[:,0,:,:]**2+flow_up_1n[:,1,:,:]**2)
            
                #bl_avg=(bl_12+bl_23+bl_34+bl_45+bl_56+bl_67)   #/7.0  +bl_78
                if j==frames-1:
                    bl_avg += 2*bl_1n
                else:
                    bl_avg += bl_1n
            bl=bl_avg.cpu().numpy().squeeze()/2/frames
       
            save_path=images[i].replace("GOPRO_Large_all/test","GTMMP/bl_test")
        
            save_path = save_path.replace("png","npy")
            
            out = bl/15  #116 154 31 67
            
            out[out>1]=1
            
            #out = out.astype(np.uint8)
            
            
            #out = cv2.equalizeHist(out)
        
            np.save(save_path, out)
        
        
            if np.max(bl)>avg_max:
                avg_max=np.max(bl)

      
    print(avg_max)

## gerate synthetic blurry frame for training

In [ ]:
pathlist=[]
pathlist.append('GTMMP/train/GOPR0372_07_00/') #7 61
pathlist.append('GTMMP/train/GOPR0372_07_01/') #7 59
pathlist.append('GTMMP/train/GOPR0374_11_00/') #11 116
pathlist.append('GTMMP/train/GOPR0374_11_01/') #11 108
pathlist.append('GTMMP/train/GOPR0374_11_02/') #11 154
pathlist.append('GTMMP/train/GOPR0374_11_03/') #11 125
pathlist.append('GTMMP/train/GOPR0378_13_00/') #13 40
pathlist.append('GTMMP/train/GOPR0379_11_00/') #11 67
pathlist.append('GTMMP/train/GOPR0380_11_00/') #11 121
pathlist.append('GTMMP/train/GOPR0384_11_01/') #11 100
pathlist.append('GTMMP/train/GOPR0384_11_02/') #11 62
pathlist.append('GTMMP/train/GOPR0384_11_03/') #11 86
pathlist.append('GTMMP/train/GOPR0384_11_04/') #11 150
pathlist.append('GTMMP/train/GOPR0385_11_00/') #11 64
pathlist.append('GTMMP/train/GOPR0386_11_00/') #11 107
pathlist.append('GTMMP/train/GOPR0477_11_00/') #11 155
pathlist.append('GTMMP/train/GOPR0857_11_00/') #11 54 
pathlist.append('GTMMP/train/GOPR0868_11_01/') #11 46
pathlist.append('GTMMP/train/GOPR0868_11_02/') #11 64
pathlist.append('GTMMP/train/GOPR0871_11_01/') #11 65
pathlist.append('GTMMP/train/GOPR0881_11_00/') #11 84
pathlist.append('GTMMP/train/GOPR0884_11_00/') #11

In [ ]:
import os
for name in pathlist:
    folder_name = root + name
    if os.path.exists(folder_name) is False:
        os.makedirs(folder_name)

In [ ]:
pathlist=[]
#path to gopro_large_all, frames used to generate a synthetic frame in original gopro dataset
pathlist.append(['GOPRO_Large_all/train/GOPR0372_07_00/',7]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0372_07_01/',7]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0374_11_00/',11])
pathlist.append(['GOPRO_Large_all/train/GOPR0374_11_01/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0374_11_02/',11])
pathlist.append(['GOPRO_Large_all/train/GOPR0374_11_03/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0378_13_00/',13]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0379_11_00/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0380_11_00/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0384_11_01/',11])
pathlist.append(['GOPRO_Large_all/train/GOPR0384_11_02/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0384_11_03/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0384_11_04/',11])
pathlist.append(['GOPRO_Large_all/train/GOPR0385_11_00/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0386_11_00/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0477_11_00/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0857_11_00/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0868_11_01/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0868_11_02/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0871_11_01/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0881_11_00/',11]) 
pathlist.append(['GOPRO_Large_all/train/GOPR0884_11_00/',11]) 

crf.npy file from [GOPRO](https://seungjunnah.github.io/Datasets/reds/crf.npy)

In [ ]:
import numpy as np
from scipy.interpolate import InterpolatedUnivariateSpline 
import glob
from PIL import Image
import os
import argparse


crf_path = 'crf.npy'
device='cpu'
C=3
H=720
W=1280
crf_inv = np.load('crf.npy').squeeze()

# 1st order approximation at RGB=250 to regularize extreme responses at RGB>250
diff = (crf_inv[251] - crf_inv[249])/2
for i in range(251, 256):
    crf_inv[i] = crf_inv[i-1] + diff

#crf_inv[:,0]=crf_inv[:,1]
#crf_inv[:,2]=crf_inv[:,1]
x=np.linspace(0,255,256)
y=crf_inv[:,0]
f1=InterpolatedUnivariateSpline(x,y)
f2=InterpolatedUnivariateSpline(y,x)
# frame interpolation, etc.

for name in pathlist:
    path = root + name[0]
    images = glob.glob(os.path.join(path, '*.png')) + \
                glob.glob(os.path.join(path, '*.jpg'))
    imglist = sorted(images)
    frames = name[1]
    imgsignal_list=[]
    print('buffering')
    for k in range(len(imglist)):
        img=Image.open(imglist[k])
        img=np.array(img)
        img=img.reshape(-1)
        img_signal=f1(img)
        imgsignal_list.append(img_signal)
    print('finish buffering')
    for i in range(0,len(imglist)-frames):
        print(i)
        img_blur_signal=0
        for j in range(frames):
            img_signal=imgsignal_list[i+j]
            img_blur_signal += img_signal
        img_blur_signal = img_blur_signal/frames
        img_blur=f2(img_blur_signal)
        img_blur=img_blur.reshape(H,W,C)
        
        outputname=imglist[i].replace("GOPRO_Large_all","GTMMP")
        img_blur=Image.fromarray(img_blur.astype('uint8'))
        img_blur.save(outputname)

## copy test files

In [14]:
import os
import shutil

root = 'E:/gopro/'

source_path = root+'GOPRO_Large/test'
target_path = root+'GTMMP/test'

if not os.path.exists(target_path):
    os.makedirs(target_path)

if os.path.exists(source_path):
    shutil.rmtree(target_path)

shutil.copytree(source_path, target_path)
print('copy dir finished!')

copy dir finished!
